# Using NEAT to evolve solutions to XOR
---

To use NEAT you must import the following.

In [1]:
from neat3 import config, population, chromosome, genome, visualize
from neat3.nn import nn_pure as nn
import pickle

### Configuring NEAT

In order to run NEAT, you'll need to configure a number of paramters.  There is a configuration file, provided below, where these parameters are set.  

The **[phenotype]** section defines the type of neural networks that will be evolved.  You need to ensure that the number of input nodes and output nodes match the requirements of your problem domain.  Typically you will start with zero hidden nodes, and let NEAT add them if necessary to achieve higher fitness. The **nn_activation** defines what activation function will be used.  The function **exp** is a sigmoid and will only return values in the range **[0,1]**.  The function **tanh** will return values in the range **[-1,1]**. Typically for robot control problems we will use **tanh**.

The **[genetic]** section defines the size of the population and the maximum fitness threshold.  Both of these should be tuned for your specific problem domain. When deciding on a **pop_size**, keep in mind that the larger the population, the longer the evolution process will take.  However, larger populations tend to yield better results.  You'll need to decide the best tradeoff of size vs speed. The **max_fitness_threshold** is used by NEAT to decide whether to stop evolution early.  If this threshold is ever achieved, NEAT will suspend evolution, otherwise it will continue until the specified number of generations is reached.

The other sections tune how NEAT operates.  I tend to leave these at the default settings, but feel free to experiment with them.

In [2]:
%%file configXOR

[phenotype]
input_nodes         = 2
output_nodes        = 1
max_weight          = 30
min_weight          = -30
feedforward         = 1
nn_activation       = exp
hidden_nodes        = 0
weight_stdev        = 0.9

[genetic]
pop_size              = 50
max_fitness_threshold = 1

# Human reasoning
prob_addconn          = 0.1
prob_addnode          = 0.05
prob_mutatebias       = 0.2
bias_mutation_power   = 0.5
prob_mutate_weight    = 0.9
weight_mutation_power = 1.5
prob_togglelink       = 0.01
elitism               = 1

[genotype compatibility]
compatibility_threshold = 3.0
compatibility_change    = 0.0
excess_coeficient       = 1.0
disjoint_coeficient     = 1.0
weight_coeficient       = 0.4

[species]
species_size        = 10
survival_threshold  = 0.2
old_threshold       = 30
youth_threshold     = 10
old_penalty         = 0.2
youth_boost         = 1.2
max_stagnation      = 15

Overwriting configXOR


### Define a fitness function

Create a fitness function that takes a chromosome as input and as a side-effect sets the chromosome's fitness. It does not return anything. A fitness function should be defined such that higher values equate with better fitness.  

For this task, we want to define fitness to match how well the network solves the XOR problem. We first calculate the sum-squared error of a network on the XOR task.  Good networks will have low error, so we will define fitness as **1 - sqrt(avgError)**.  In order to evaluate a NEAT chromosome, you must first convert the chromosome into a phenotype (a network).  Then you can activate the network and test its output values.

In [3]:
import math
INPUTS = [[0, 0], [0, 1], [1, 0], [1, 1]]
OUTPUTS = [0, 1, 1, 0]

def eval_individual(chromo):
    net = nn.create_ffphenotype(chromo)
    error = 0.0
    for i, inputs in enumerate(INPUTS):
        net.flush() 
        output = net.sactivate(inputs)
        error += (output[0] - OUTPUTS[i])**2
    chromo.fitness = 1 - math.sqrt(error/len(OUTPUTS))

### Define a population evaluation function

NEAT requires a population evaluation function that takes a population as input. A population is defined to be a list of chromosomes.  This function will be called once for every generation of the evolution, and it will evaluate the fitness of every member of the population.

In [4]:
def eval_population(population):
    for chromo in population:
        eval_individual(chromo)

### Evolve

The evolve function takes the number of generations to run. Each generation, NEAT will print out statistics about the population, including the current best fitness and the average fitness. 

In [5]:
def evolve(n):
    config.load("configXOR")
    chromosome.node_gene_type = genome.NodeGene

    # Tell NEAT that we want to use the above function to evaluate fitness
    population.Population.evaluate = eval_population

    # Create a population (the size is defined in the configuration file)
    pop = population.Population()

    # Run NEAT's genetic algorithm for at most 30 epochs
    # It will stop if fitness surpasses the max_fitness_threshold in config file
    pop.epoch(n, report=True, save_best=True, name="XOR")

    # Plots the evolution of the best/average fitness
    visualize.plot_stats(pop.stats)

    # Visualizes speciation
    visualize.plot_species(pop.species_log)

In [6]:
evolve(30)


 ****** Running generation 0 ****** 

Population's average fitness: 0.39660 stdev: 0.06082
Best fitness: 0.4998555915 - size: (0, 2) - species 1 - id 30
Species length: 1 totalizing 50 individuals
Species ID       : [1]
Each species size: [50]
Amount to spawn  : [50]
Species age      : [0]
Species no improv: [0]

 ****** Running generation 1 ****** 

Population's average fitness: 0.40501 stdev: 0.06442
Best fitness: 0.4998555915 - size: (0, 2) - species 1 - id 30
Species length: 1 totalizing 50 individuals
Species ID       : [1]
Each species size: [50]
Amount to spawn  : [50]
Species age      : [1]
Species no improv: [0]

 ****** Running generation 2 ****** 

Population's average fitness: 0.39970 stdev: 0.07279
Best fitness: 0.4998555915 - size: (0, 2) - species 1 - id 30
Species length: 3 totalizing 50 individuals
Species ID       : [1, 2, 3]
Each species size: [47, 2, 1]
Amount to spawn  : [17, 14, 20]
Species age      : [2, 0, 0]
Species no improv: [1, 0, 0]
Removing 1 excess indiv

Species no improv: [6, 6, 0, 4, 1, 1, 3, 0, 0]
Selecting 1 more indiv(s) to fill up the new population

 ****** Running generation 25 ****** 

Population's average fitness: 0.44886 stdev: 0.09305
Best fitness: 0.6260981886 - size: (1, 4) - species 1 - id 1064
Species length: 9 totalizing 50 individuals
Species ID       : [1, 4, 6, 7, 12, 13, 14, 15, 16]
Each species size: [4, 6, 9, 6, 12, 6, 5, 1, 1]
Amount to spawn  : [6, 5, 5, 6, 7, 6, 7, 4, 4]
Species age      : [25, 19, 15, 14, 5, 4, 4, 1, 1]
Species no improv: [7, 7, 1, 5, 2, 2, 4, 1, 1]

 ****** Running generation 26 ****** 

Population's average fitness: 0.45017 stdev: 0.10080
Best fitness: 0.6260981886 - size: (1, 4) - species 1 - id 1064
Species length: 8 totalizing 50 individuals
Species ID       : [1, 4, 6, 7, 12, 13, 15, 16]
Each species size: [6, 12, 5, 6, 7, 6, 4, 4]
Amount to spawn  : [7, 6, 7, 7, 7, 8, 5, 5]
Species age      : [26, 20, 16, 15, 6, 5, 2, 2]
Species no improv: [8, 8, 2, 6, 3, 3, 0, 2]
Removing 2 excess ind

### Evaluating the results of evolution

NEAT will save several plots, one showing the best and average fitness over time, and another showing how NEAT's population speciated over time. These plots will have **.svg** extensions and can be opened from a terminal window using **eog**. Try viewing these plots.

In addition, NEAT will save the best chromosome present in the population at the end of each generation. To test a particular chromosome file, you can use the following function. This will also generate a new **.svg** file that includes a visualization of the network's topology.  Try viewing this plot which will have a filename that begins with **phenotype**.

In [7]:
def eval_best(chromo_file):
    fp = open(chromo_file, "rb")
    chromo = pickle.load(fp)
    fp.close()
    print(chromo)
    visualize.draw_net(chromo, "_"+ chromo_file)
    
    print('\nNetwork output:')
    brain = nn.create_ffphenotype(chromo)
    for i, inputs in enumerate(INPUTS):
        output = brain.sactivate(inputs)
        print("%1.5f \t %1.5f" %(OUTPUTS[i], output[0]))

In [8]:
eval_best("XOR_best_chromo_29")

Nodes:
	Node  1  INPUT, bias  0, response 4.924273
	Node  2  INPUT, bias  0, response 4.924273
	Node  3 OUTPUT, bias 1.56438882, response 4.924273
	Node  4 HIDDEN, bias 0.95296582, response 4.94640357
Connections:
	In  1, Out  3, Weight +0.40704, Enabled, Innov 1
	In  2, Out  3, Weight +0.67050, Disabled, Innov 2
	In  2, Out  4, Weight -2.18989, Enabled, Innov 3
	In  4, Out  3, Weight -1.88029, Enabled, Innov 4
	In  1, Out  4, Weight +3.32298, Enabled, Innov 5
Node order: [4]

Network output:
0.00000 	 0.18645
1.00000 	 0.99954
1.00000 	 0.61036
0.00000 	 0.61043


### Experiments

Run 10 experiments on the XOR problem using a popultion of size 50.  For each experiment, record whether NEAT evolved a successful solution, and the number of hidden nodes it generated.  

Run 10 more experiments on the XOR problem using a different population size (you can try smaller or larger, whichever you prefer).  Record the same information as before, and compare the results.

### Use git to commit, add, and push

Save this notebook before moving on.